In [2]:
from datasets import load_dataset

ds = load_dataset("hackercupai/hackercup")
ds

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    sample: Dataset({
        features: ['name', 'year', 'round', 'statement', 'input', 'solution', 'code', 'output', 'sample_input', 'sample_output', 'images'],
        num_rows: 10
    })
    full: Dataset({
        features: ['name', 'year', 'round', 'statement', 'input', 'solution', 'code', 'output', 'sample_input', 'sample_output', 'images'],
        num_rows: 284
    })
})

In [6]:
ds['sample']['name']

['meta_game',
 'tower_rush',
 'ready_go_part_2',
 'wiki_race',
 'ready_go_part_1',
 'dim_sum_delivery',
 'cheeseburger_corollary_ch1',
 'two_apples_a_day',
 'cheeseburger_corollary_ch2',
 'road_to_nutella']

In [23]:
import os
import json
import contextlib
import io
from unittest.mock import patch
import time
import re
import traceback

def check_code_structure(extracted_code):
    # Check if the phrase "__name__ == '__main__'" is present in the code
    if "__name__ == '__main__'" not in extracted_code:
        return False, "Missing `if __name__ == '__main__':` block."
    return True, None

def run_extracted_code(extracted_code, test_input):
    # Check if the structure of the code is valid
    is_valid, error_message = check_code_structure(extracted_code)
    
    # Output and Error containers
    output = io.StringIO()
    error = None
    test_input_lines = [line.strip() for line in test_input.strip().split('\n') if line.strip()]
    
    def mock_input():
        if not test_input_lines:
            raise ValueError("Not enough input data provided")
        return test_input_lines.pop(0)

    with patch('builtins.input', mock_input), contextlib.redirect_stdout(output):
        try:
            # Compile the code object
            code_obj = compile(extracted_code, '<string>', 'exec')
            local_scope = {'__name__': '__main__'}
            exec(code_obj, local_scope)

        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            
            # Get the line number where the error occurred
            tb = traceback.extract_tb(exc_traceback)
            line_no = tb[-1].lineno
            
            # Get the line of code that caused the error
            code_lines = extracted_code.split('\n')
            error_line = code_lines[line_no - 1] if line_no <= len(code_lines) else "Unknown"
            
            error = f"Error occurred on line {line_no}: "
            error += f"{error_line.strip()}\n"
            error += f"Exception: {exc_type.__name__}: {str(exc_value)}\n"
            return None, error

    result = output.getvalue()
    
    # Check if the result is empty and return an appropriate error
    if not result.strip():
        return None, "Error: Code did not produce any output."

    return result, error

# Compare the output generated by the code with the expected output
def compare_with_expected_output(generated_output, expected_output):
    # Check if either output is None
    if generated_output is None:
        return 0, ["Generated output is None"]

    generated_output_lines = generated_output.strip().splitlines()
    expected_output_lines = expected_output.strip().splitlines()

    total_cases = len(expected_output_lines)
    matching_cases = 0
    failed_cases = []

    for i, (generated_line, expected_line) in enumerate(zip(generated_output_lines, expected_output_lines), start=1):
        if generated_line.strip() == expected_line.strip():
            matching_cases += 1
        else:
            failed_cases.append(f"Test Case #{i}: Expected '{expected_line}' but got '{generated_line}'")

    score = (matching_cases / total_cases) * 100 if total_cases > 0 else 0
    return score, failed_cases

# Evaluate generated code on test cases
def evaluate_generated_code_on_test_cases(extracted_code, test_input, test_output):
    # Run the code and get the output
    generated_output, error = run_extracted_code(extracted_code, test_input)
    
    if generated_output is None or generated_output.strip() == "":
        return 0, error or "Error: The code ran without any problem. There's no error in parsing the input. But it produces no output. Please check the entire code again.", generated_output, []
    
    # If there's an error, return it
    if error:
        return 0, error, generated_output, []

    # Compare the generated output with expected output
    score, failed_cases = compare_with_expected_output(generated_output, test_output)
    
    if failed_cases:
        error_msg = f"Test cases failed: {failed_cases}"
        return score, error_msg, generated_output, failed_cases

    return score, error, generated_output, failed_cases

In [24]:
# Sample Python code for evaluation
sample_code = """
def add_numbers(a, b):
    return a + b

if __name__ == '__main__':
    num1 = int(input())
    num2 = int(input())
    result = add_numbers(num1, num2)
    print(result)
"""

# Sample test cases with inputs and expected outputs
test_cases = [
    {"input": "3\n5\n", "expected_output": "8\n"},
    {"input": "10\n20\n", "expected_output": "30\n"},
    {"input": "-4\n6\n", "expected_output": "2\n"}
]

# Evaluating each test case
for i, test_case in enumerate(test_cases, start=1):
    test_input = test_case["input"]
    expected_output = test_case["expected_output"]
    score, error, generated_output, failed_cases = evaluate_generated_code_on_test_cases(
        sample_code, test_input, expected_output
    )
    
    # Printing the results for each test case
    print(f"Test Case #{i}:")
    if error:
        print(f"Error: {error}")
    else:
        print(f"Score: {score}%")
        print(f"Generated Output: {generated_output.strip()}")
    print("-" * 40)


Test Case #1:
Score: 100.0%
Generated Output: 8
----------------------------------------
Test Case #2:
Score: 100.0%
Generated Output: 30
----------------------------------------
Test Case #3:
Score: 100.0%
Generated Output: 2
----------------------------------------


: 

In [20]:
code_string = """
def find_root(parent, i):
    while parent[i] != i:
        i = parent[i]
    return i

def union_sets(parent, rank, x, y):
    rootX = find_root(parent, x)
    rootY = find_root(parent, y)
    if rootX != rootY:
        if rank[rootX] > rank[rootY]:
            parent[rootY] = rootX
        elif rank[rootX] < rank[rootY]:
            parent[rootX] = rootY
        else:
            parent[rootY] = rootX
            rank[rootX] += 1

def kruskal_mst(edges, num_vertices):
    edges.sort(key=lambda item: item[2])
    parent = list(range(num_vertices))
    rank = [0] * num_vertices
    mst_edges = []
    mst_weight = 0

    for edge in edges:
        u, v, weight = edge
        rootU = find_root(parent, u)
        rootV = find_root(parent, v)

        if rootU != rootV:
            mst_edges.append(edge)
            mst_weight += weight
            union_sets(parent, rank, rootU, rootV)

    return mst_weight, mst_edges

def solve(input_string):
    input_lines = input_string.strip().split('\\n')
    num_vertices, num_edges = map(int, input_lines[0].split())
    edges = []

    for i in range(1, num_edges + 1):
        u, v, weight = map(int, input_lines[i].split())
        edges.append((u, v, weight))

    mst_weight, mst_edges = kruskal_mst(edges, num_vertices)
    output = f"Minimum Spanning Tree weight: {mst_weight}\\nEdges in MST:\\n"
    for u, v, weight in mst_edges:
        output += f"{u} - {v} : {weight}\\n"

    return output

# Sample input
sample_input = \"\"\"4 5
0 1 10
0 2 6
0 3 5
1 3 15
2 3 4
\"\"\"

# Execute the solve function
sample_output = solve(sample_input)
print(sample_output)
"""



In [21]:
# Dictionary to hold the local scope
local_scope = {}

# Execute the code in the local scope
exec(code_string, globals(), local_scope)

# Now you can use functions from the local scope dictionary
sample_input = """4 5
0 1 10
0 2 6
0 3 5
1 3 15
2 3 4
"""

# Call the solve function from the local scope
sample_output = local_scope['solve'](sample_input)
print(sample_output)


Minimum Spanning Tree weight: 19
Edges in MST:
2 - 3 : 4
0 - 3 : 5
0 - 1 : 10

Minimum Spanning Tree weight: 19
Edges in MST:
2 - 3 : 4
0 - 3 : 5
0 - 1 : 10



In [22]:
local_scope

{'find_root': <function __main__.find_root(parent, i)>,
 'union_sets': <function __main__.union_sets(parent, rank, x, y)>,
 'kruskal_mst': <function __main__.kruskal_mst(edges, num_vertices)>,
 'solve': <function __main__.solve(input_string)>,
 'sample_input': '4 5\n0 1 10\n0 2 6\n0 3 5\n1 3 15\n2 3 4\n',
 'sample_output': 'Minimum Spanning Tree weight: 19\nEdges in MST:\n2 - 3 : 4\n0 - 3 : 5\n0 - 1 : 10\n'}

In [3]:
import os 

os.listdir(".adapter/")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '.adapter/'